In [ ]:
import yfinance as yf

bpac3 = yf.Ticker("BPAC3.SA")
df_bpac3 = bpac3.history(start="2021-01-01",end="2023-12-30")

In [1]:

from sklearn.metrics import root_mean_squared_error
import pandas as pd
import plotly.express as px

matrix = {"janela":[], "rmse_a": [], "rmse_e":[]}
for i in range(2, 15):

    name_a = f'med_mov_aritimética_{i}'
    name_e = f'med_mov_exponencial_{i}'
    df_bpac3[name_a] = df_bpac3['Close'].rolling(window=i, min_periods=0).mean()
    df_bpac3[name_e] = df_bpac3['Close'].ewm(span=i, min_periods=0,adjust=False).mean()
    rmse_a = root_mean_squared_error(df_bpac3['Close'].to_numpy(), df_bpac3[name_a].to_numpy())
    rmse_e = root_mean_squared_error(df_bpac3['Close'].to_numpy(), df_bpac3[name_e].to_numpy())
    matrix["janela"].append(i)
    matrix["rmse_a"].append(rmse_a.round(5))
    matrix["rmse_e"].append(rmse_e.round(5))



ModuleNotFoundError: No module named 'data'

In [ ]:


df = pd.DataFrame(matrix)

fig = px.line(
    df, 
    x='janela',
    y=['rmse_a', 'rmse_e'],
    
)
fig.update_layout(title="Relação RMSE x Janela de Médias Móveis para BPAC3", xaxis_title="Janelas", yaxis_title="RMSE") 

fig.show()